# Noise learning helper

The error mitigation techniques [PEA](./error-mitigation-and-suppression-techniques#probabilistic-error-amplification-pea) and [PEC](./error-mitigation-and-suppression-techniques#probabilistic-error-cancellation-pec) both utilize a noise learning component based on a [Pauli-Lindblad noise model](https://arxiv.org/abs/2201.09866), which is typically managed during execution after submitting one or more jobs through `qiskit-ibm-runtime`. However, versions of `qiskit-ibm-runtime` 0.27.1 and later include a [`NoiseLearner`](../api/qiskit-ibm-runtime/noise_learner) and associated  `NoiseLearnerOptions` class for executing and storing the results of these noise learning experiments. This page provides an overview of its usage and the associated options available.


## Overview

The `NoiseLearner` class performs experiments which characterizes noise processes based on a Pauli-Lindblad noise model for one (or more) circuits. It possesses a `run()` method which will execute the learning experiments and takes as input either a list of circuits or a [PUB](./primitive-input-output#overview-of-pubs) and returns a `NoiseLearnerResult` containing the learned noise channels and metadata about the job(s) submitted. Below is a code snippet demonstrating the usage of the helper program.

In [ ]:
from qiskit import QuantumRegister, QuantumCircuit
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager

from qiskit_ibm_runtime import QiskitRuntimeService
from qiskit_ibm_runtime.noise_learner import NoiseLearner
from qiskit_ibm_runtime.options import NoiseLearnerOptions

# Build a simple GHZ circuit with three entangling layers
circuit_to_learn = QuantumCircuit(3)
circuit_to_learn.h(0)

circuit_to_learn.cx(0, 1)
circuit_to_learn.cx(1, 2)
circuit_to_learn.cx(0, 1)

# Choose a backend to run on
service = QiskitRuntimeService()
backend = service.least_busy()

# Transpile the circuit for execution
pm = generate_preset_pass_manager(backend=backend, optimization_level=3)
circuit_to_run = pm.run(circuit_to_learn)

# Instantiate a NoiseLearner object and execute the noise learning program
learner = NoiseLearner(mode=backend)
job = learner.run([circuit_to_run])
result = job.result()

The resulting `NoiseLearnerResult.data` is a list of [`LayerError`](../api/qiskit-ibm-runtime/qiskit_ibm_runtime.utils.noise_learner_result.LayerError) objects containing the [noise model](https://arxiv.org/abs/2201.09866) for each individual entangling layer that belongs to the target circuit(s). Each `LayerError` stores the layer information, in the form of a circuit and a set of qubit labels, alongside the [`PauliLindBladError`](../api/qiskit-ibm-runtime/qiskit_ibm_runtime.utils.noise_learner_result.PauliLindbladError) for the noise model that was learned for the given layer.

In [5]:
print(f'Noise learner result contains {len(result.data)} entries and has the following data:\n {result}\n')
print(f'The layer error model for the first layer has data:\n {result.data[0]}\n')
print(f'And the Pauli Lindblad model has has the following data: \n{result.data[0].error}\n')

Noise learner result contains 3 entries and has the following data:
 NoiseLearnerResult(data=[LayerError(circuit=<qiskit.circuit.quantumcircuit.QuantumCircuit object at 0x7f5140f739d0>, qubits=[0, 1], error=PauliLindbladError(generators=['IX', 'IY', 'IZ', 'XI', 'XX', 'XY', 'XZ', 'YI', 'YX', 'YY', 'YZ', 'ZI',
 'ZX', 'ZY', 'ZZ'], rates=[0.00037, 0.00046, 0.00035, 0.00127, 0.00067, 0.0, 0.0, 0.00067, 0.00127, 0.0, 0.0, 0.00071, 0.0, 0.00035, 0.00046]))), LayerError(circuit=<qiskit.circuit.quantumcircuit.QuantumCircuit object at 0x7f5140f73410>, qubits=[0, 1, 2], error=PauliLindbladError(generators=['IIX', 'IIY', 'IIZ', 'IXI', 'IXX', 'IXY', 'IXZ', 'IYI', 'IYX', 'IYY',
 'IYZ', 'IZI', 'IZX', 'IZY', 'IZZ', 'XII', 'XXI', 'XYI', 'XZI', 'YII',
 'YXI', 'YYI', 'YZI', 'ZII', 'ZXI', 'ZYI', 'ZZI'], rates=[0.00068, 0.00023, 0.00041, 0.00093, 0.00056, 0.00034, 0.0001, 0.00023, 0.00126, 0.0, 0.0, 0.00094, 5e-05, 0.00041, 0.00023, 0.00093, 0.0, 0.0, 0.0, 0.00259, 9e-05, 9e-05, 0.0, 0.01374, 0.00024, 0.00

The `LayerError.error` attribute of the noise learning result contains the generators and error rates of the fitted Pauli Lindblad model, which has the form:

$$ \Lambda(\rho) = \exp{\sum_j r_j \left(P_j \rho P_j^\dagger - \rho\right)} $$

where the $r_j$ are the `LayerError.rates` and $P_j$ are the Pauli operators specified in `LayerError.generators`.

## Noise learning options

There are a few options available to input into a `NoiseLearner` object, can be set with when instantiated. These are encapsulated by the `qiskit_ibm_runtime.options.NoiseLearnerOptions` class and include:

- `max_layers_to_learn`: The maximum number of unique entangling layers to learn (the default value is `4` and can be set to `None` to indicate no limit on the number of layers)
- `num_randomizations`: Total number of random circuits to use per learning circuit configuration
- `shots_per_randomization`: Total number of shots per learning circuit
- `layer_pair_depths`: The circuit depths to us in the learning experiments (measured in number of pairs)
- `twirling_strategy`: The twirling strategy used in the layers of two-qubit twirled gates. The allowed values are:
    - `"active"`: only instruction qubits are twirled in each individual twirled layer
    - `"active-circuit"`: in each twirled layer, the union of all qubits with an instruction are twirled
    - `"active-accum"`: in each twirled layer, the union of all qubits with an instruction in the circuit *up to the current layer* are twirled
    - `"all"`: in each twirled layer, *all* qubits in the circuit are twirled

Below is a simple example showing how to use the `NoiseLearnerOptions` in a `NoiseLeaner` experiment:

In [ ]:
# Build a larger GHZ circuit
circuit = QuantumCircuit(10)
circuit.h(0)
circuit.cx(range(0, 10, 2), range(1, 10, 2))
# Choose a backend to run on
service = QiskitRuntimeService()
backend = service.least_busy()

# Transpile the circuit for execution
pm = generate_preset_pass_manager(backend=backend, optimization_level=3)
circuit_to_run = pm.run(circuit_to_learn)

# Instantiate a noise learner options object
learner_options = NoiseLearnerOptions(
                    max_layers_to_learn = 3,
                    num_randomizations = 32,
                    twirling_strategy = "all"
                )

# Instantiate a NoiseLearner object and execute the noise learning program
learner = NoiseLearner(mode=backend, options=learner_options)
job = learner.run([circuit_to_run])
result = job.result()